In [1]:
import sys
# append path here so that jupyter can find it. As alternative you can configure this in the setting of your IDE.
sys.path.append(r"C:\Users\oliver.koehn\Documents\gitProjects\qualityInspection")

from lib.utils import *
import pandas as pd

# PARAM
connection_string = os.getenv("CSTR__BLOB__VISINSP")
container_name = "image-uploads"
fp__img__blob = "2024/05/21/cast_ok_0_525.jpeg"
log_schema = "VISUAL_INSPECTION"
log_table = "LOG__VISUAL_INSPECTION"

# DB CONNECTION
eng = get_engine()

# BLOB CONNECTION
blob_service_client = get_blob_service_client(connection_string)

# EVAL
loaded_model = get_keras_model(blob_service_client, "local", "trained_model.keras")
prob__ok = eval_img_from_blob(loaded_model, blob_service_client, container_name, fp__img__blob)

# LOG TO DB
j__db = {"FN__BLOB" : fp__img__blob.split("/")[-1], "CONTAINER_NAME" : container_name, "PROBABILITY_OK": round(prob__ok, 4), "FP__BLOB" : fp__img__blob}
df = pd.DataFrame(j__db, index=[0])
df.to_sql(log_table, eng, schema=log_schema, if_exists="append", index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


1

In [ ]:
sql = f"SELECT * FROM {log_schema}.{log_table};"
df = pd.read_sql(sql, eng)

df

In [ ]:
"""
/*
create schema VISUAL_INSPECTION;
DROP TABLE VISUAL_INSPECTION.LOG__VISUAL_INSPECTION;
*/
CREATE TABLE VISUAL_INSPECTION.LOG__VISUAL_INSPECTION
(
    FN__BLOB VARCHAR(200)
    , CONTAINER_NAME VARCHAR(200)
    , PROBABILITY_OK float
    , FP__BLOB VARCHAR(200)
    , PRIMARY KEY (FN__BLOB)
)
;
"""